<a href="https://colab.research.google.com/github/aniketSanyal/OverfittingInRML/blob/main/rml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [90]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

# Define Models for CIFAR10 and MNIST

In [91]:
class CIFAR10_CNN(nn.Module):
    def __init__(self):
        super(CIFAR10_CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = F.relu(self.pool(self.conv1(x)))
        x = F.relu(self.pool(self.conv2(x)))
        x = F.relu(self.pool(self.conv3(x)))
        x = x.view(-1, 128 * 4 * 4)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class MNIST_CNN(nn.Module):
    def __init__(self):
        super(MNIST_CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 7 * 7, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = F.relu(self.pool(self.conv1(x)))
        x = F.relu(self.pool(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# Define FGSM and PGD Attack Functions

In [92]:
def fgsm_attack(image, epsilon, data_grad):
    # Collect the element-wise sign of the data gradient
    sign_data_grad = data_grad.sign()
    # Create the perturbed image by adjusting each pixel of the input image
    perturbed_image = image + epsilon * sign_data_grad
    # Adding clipping to maintain [0,1] range
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    return perturbed_image

def pgd_attack(model, image, label, epsilon, alpha, iters, device):
    # Initialize perturbation as zero
    perturbation = torch.zeros_like(image).to(device)
    perturbation.requires_grad = True

    for _ in range(iters):
        outputs = model(image + perturbation)
        loss = F.cross_entropy(outputs, label)
        model.zero_grad()
        loss.backward()

        # Update the perturbation
        perturbation.data += alpha * perturbation.grad.data.sign()
        perturbation.data = torch.clamp(perturbation.data, -epsilon, epsilon)

    # Apply the perturbation and clip the result
    perturbed_image = torch.clamp(image + perturbation, 0, 1)
    return perturbed_image


# Load Data for Both Datasets

In [93]:
# Transformations for CIFAR10
transform_cifar = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalizing for CIFAR10
])

# Transformations for MNIST
transform_mnist = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalizing for MNIST
])

# CIFAR10
cifar_train = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_cifar)
cifar_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_cifar)

# MNIST
mnist_train = datasets.MNIST(root='./data', train=True, download=True, transform=transform_mnist)
mnist_test = datasets.MNIST(root='./data', train=False, download=True, transform=transform_mnist)

# Data loaders with batch processing
batch_size = 64  # You can adjust the batch size

cifar_train_loader = DataLoader(cifar_train, batch_size=batch_size, shuffle=True)
cifar_test_loader = DataLoader(cifar_test, batch_size=batch_size, shuffle=False)

mnist_train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True)
mnist_test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


# Training and Testing Functions

In [94]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()  # Set the model to training mode
    total_loss = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()  # Zero the gradients
        output = model(data)  # Forward pass
        loss = F.cross_entropy(output, target)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

        total_loss += loss.item()
        if batch_idx % 100 == 0:  # Print status every 100 batches
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

    average_loss = total_loss / len(train_loader.dataset)
    print(f'Training set: Average loss: {average_loss:.4f}')

def test(model, device, test_loader):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0
    correct = 0

    with torch.no_grad():  # No need to track gradients for testing
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()  # Sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)

    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.0f}%)\n')

def train_adversarial(model, device, train_loader, optimizer, epoch, attack, epsilon=0.1, alpha=None, iters=None):
    model.train()
    total_loss = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        # Generate adversarial examples
        if attack == fgsm_attack:
            data.requires_grad = True
            output = model(data)
            loss = F.cross_entropy(output, target)
            model.zero_grad()
            loss.backward()
            data_grad = data.grad.data
            adversarial_data = fgsm_attack(data, epsilon, data_grad)
        elif attack == pgd_attack and alpha is not None and iters is not None:
            adversarial_data = pgd_attack(model, data, target, epsilon, alpha, iters, device)
        else:
            raise ValueError("Invalid attack method or missing parameters for PGD")

        # Training step with adversarial examples
        optimizer.zero_grad()
        output = model(adversarial_data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

    average_loss = total_loss / len(train_loader.dataset)
    print(f'Training set: Average loss: {average_loss:.4f}')

def test_adversarial(model, device, test_loader, attack, epsilon=0.1, alpha=None, iters=None):
    model.eval()
    test_loss = 0
    correct = 0

    for data, target in test_loader:
        data, target = data.to(device), target.to(device)

        # Set requires_grad attribute of tensor. Important for Attack
        data.requires_grad = True

        # Generate adversarial examples
        if attack == fgsm_attack:
            output = model(data)
            loss = F.cross_entropy(output, target)
            model.zero_grad()
            loss.backward()
            data_grad = data.grad.data
            adversarial_data = fgsm_attack(data, epsilon, data_grad)
        elif attack == pgd_attack and alpha is not None and iters is not None:
            adversarial_data = pgd_attack(model, data, target, epsilon, alpha, iters, device)
        else:
            raise ValueError("Invalid attack method or missing parameters for PGD")

        # Evaluate on adversarial examples
        output = model(adversarial_data)
        test_loss += F.cross_entropy(output, target, reduction='sum').item()  # Sum up batch loss
        pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'\nTest set (adversarial): Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.0f}%)\n')


# Training and Testing with Adversarial Attacks

In [95]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize CIFAR10 and MNIST models
model_cifar = CIFAR10_CNN().to(device)
model_mnist = MNIST_CNN().to(device)

# Define optimizers for each model
optimizer_cifar = torch.optim.Adam(model_cifar.parameters(), lr=0.001)
optimizer_mnist = torch.optim.Adam(model_mnist.parameters(), lr=0.001)

num_epochs = 10  # Define the number of epochs
# Set parameters for attacks
epsilon = 0.1  # Perturbation magnitude for FGSM
alpha = 0.01   # Step size for PGD
iters = 40     # Number of iterations for PGD

for epoch in range(num_epochs):
    # Training with original data
    train(model_cifar, device, cifar_train_loader, optimizer_cifar, epoch)
    train(model_mnist, device, mnist_train_loader, optimizer_mnist, epoch)

    # Training with FGSM adversarial examples
    train_adversarial(model_cifar, device, cifar_train_loader, optimizer_cifar, epoch, fgsm_attack, epsilon)
    train_adversarial(model_mnist, device, mnist_train_loader, optimizer_mnist, epoch, fgsm_attack, epsilon)

    # Training with PGD adversarial examples
    train_adversarial(model_cifar, device, cifar_train_loader, optimizer_cifar, epoch, pgd_attack, epsilon, alpha, iters)
    train_adversarial(model_mnist, device, mnist_train_loader, optimizer_mnist, epoch, pgd_attack, epsilon, alpha, iters)


# Test on clean examples
test(model_cifar, device, cifar_test_loader)
test(model_mnist, device, mnist_test_loader)

# FGSM attack parameters
fgsm_epsilon = 0.1
# Test on FGSM adversarial examples
test_adversarial(model_cifar, device, cifar_test_loader, fgsm_attack, epsilon=fgsm_epsilon)
test_adversarial(model_mnist, device, mnist_test_loader, fgsm_attack, epsilon=fgsm_epsilon)

# PGD attack parameters
pgd_epsilon = 0.1  # Perturbation magnitude
pgd_alpha = 0.01   # Step size
pgd_iters = 40     # Number of iterations

# Test on PGD adversarial examples
test_adversarial(model_cifar, device, cifar_test_loader, pgd_attack, epsilon=pgd_epsilon, alpha=pgd_alpha, iters=pgd_iters)
test_adversarial(model_mnist, device, mnist_test_loader, pgd_attack, epsilon=pgd_epsilon, alpha=pgd_alpha, iters=pgd_iters)


Train Epoch: 0 [0/50000 (0%)]	Loss: 2.300543
Train Epoch: 0 [6400/50000 (13%)]	Loss: 1.713624
Train Epoch: 0 [12800/50000 (26%)]	Loss: 1.654285
Train Epoch: 0 [19200/50000 (38%)]	Loss: 1.329940
Train Epoch: 0 [25600/50000 (51%)]	Loss: 1.075407
Train Epoch: 0 [32000/50000 (64%)]	Loss: 1.483519
Train Epoch: 0 [38400/50000 (77%)]	Loss: 1.235259
Train Epoch: 0 [44800/50000 (90%)]	Loss: 0.979644
Training set: Average loss: 0.0214
Train Epoch: 0 [0/60000 (0%)]	Loss: 2.305349
Train Epoch: 0 [6400/60000 (11%)]	Loss: 0.225620
Train Epoch: 0 [12800/60000 (21%)]	Loss: 0.096083
Train Epoch: 0 [19200/60000 (32%)]	Loss: 0.140489
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.128525
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.045435
Train Epoch: 0 [38400/60000 (64%)]	Loss: 0.014105
Train Epoch: 0 [44800/60000 (75%)]	Loss: 0.039597
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.106394
Train Epoch: 0 [57600/60000 (96%)]	Loss: 0.009208
Training set: Average loss: 0.0022
Train Epoch: 0 [0/50000 (0%)]	Loss: 2.9194

# Visualizing the Results